In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os


In [2]:
movies_data =pd.read_csv(r'Downloads/movies.csv')
movies_data.head()


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
display (movies_data.shape)


(4803, 24)

In [4]:
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)


['genres', 'keywords', 'tagline', 'cast', 'director']


In [5]:
display (movies_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

None

In [6]:
display (movies_data.isna().sum())


index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [7]:
display (movies_data[selected_features].head())


,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [8]:
display (movies_data[selected_features].isna().sum())

genres       28
keywords    412
tagline     844
cast         43
director     30
dtype: int64

In [9]:
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')
display (movies_data.head())



,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [10]:
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']
display (combined_features)


0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [11]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
display (feature_vectors.shape)
print (feature_vectors)


(4803, 17318)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [12]:
similarity = cosine_similarity(feature_vectors)
print  (similarity )


[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [13]:
display(similarity.shape)


(4803, 4803)

In [14]:
movie_name = input(' Enter your favourite movie name : ')


 Enter your favourite movie name : Sita Ramam


In [15]:

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)


['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [16]:
len(list_of_all_titles)

4803

In [17]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)


[]


In [18]:
close_match = find_close_match[0]
print(close_match)


IndexError: list index out of range

In [22]:


# Example variables (replace these with actual data)
movie_name = "sanam teri kasam"
list_of_all_titles = ["sanam teri kasam", "Sita Ramam", "Shadi me jaroor aana"]

# Find close matches
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

# Check if there are any close matches before accessing the first one
if find_close_match:
    close_match = find_close_match[0]
    print(close_match)
else:
    print("No close matches found.")


['sanam teri kasam']
sanam teri kasam


In [21]:
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [24]:



# Example data (replace this with your actual data)
movies_data = pd.DataFrame({
    'title': ["sanam teri kasam", "Sita Ramam", "Shadi me jaroor aana"],
    'index': [1, 2, 3]
})

# Example variables (replace these with actual data)
movie_name = "sanam teri kasam"
list_of_all_titles = movies_data['title'].tolist()

# Find close matches
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

# Check if there are any close matches before proceeding
if find_close_match:
    close_match = find_close_match[0]
    print(close_match)
    
    # Find the index of the movie
    movie_indices = movies_data[movies_data.title == close_match]['index'].values
    
    if movie_indices.size > 0:
        index_of_the_movie = movie_indices[0]
        print(index_of_the_movie)
    else:
        print("No index found for the matched movie.")
else:
    print("No close matches found.")


['sanam teri kasam']
sanam teri kasam
1


In [25]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)


[(0, 0.07219486822992491), (1, 1.0000000000000004), (2, 0.032814990806793745), (3, 0.020741502866825225), (4, 0.03305463079248628), (5, 0.09894823392218867), (6, 0.014378790821108607), (7, 0.019781895260853298), (8, 0.04056180271185782), (9, 0.029948196927440142), (10, 0.06512426368922568), (11, 0.027580931446803184), (12, 0.5325423014890582), (13, 0.16315140691538466), (14, 0.05752279793612993), (15, 0.02384630652060951), (16, 0.018757441743698592), (17, 0.08693929581992486), (18, 0.017747962646584), (19, 0.061598207762639565), (20, 0.08780812424608472), (21, 0.016447740747051347), (22, 0.0473755607656848), (23, 0.044397771855870655), (24, 0.07038585257947678), (25, 0.0), (26, 0.02041663626765186), (27, 0.1458486783441737), (28, 0.06070748232215177), (29, 0.020505067521231088), (30, 0.09738646725014082), (31, 0.045173032380404654), (32, 0.08732451789987261), (33, 0.019658984415172717), (34, 0.0), (35, 0.020637015141285384), (36, 0.018595597513519024), (37, 0.026261794048384475), (38, 

In [26]:
len(similarity_score)

4803

In [27]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)


[(1, 1.0000000000000004), (199, 0.5466481253706559), (12, 0.5325423014890582), (536, 0.288818238643064), (1652, 0.19164772894913185), (384, 0.18874711394669408), (13, 0.16315140691538466), (129, 0.15873396251727565), (3055, 0.15577364618957568), (3033, 0.15459290728645847), (329, 0.15057603842736808), (178, 0.14725915063827233), (27, 0.1458486783441737), (2035, 0.14149462529605056), (267, 0.14000715404465205), (1183, 0.13987145541658172), (326, 0.13846182473796617), (1095, 0.13674871062299218), (1890, 0.13658339038642311), (2401, 0.13627388719755615), (2040, 0.13546842154573419), (2474, 0.13515170184139358), (2254, 0.13504511148016585), (1577, 0.12992151193805673), (894, 0.12882247930032706), (1061, 0.1285840875411398), (189, 0.12748140950217185), (2515, 0.1263816681891445), (2815, 0.12602945523943587), (1063, 0.125934979358607), (1635, 0.12565457498567212), (840, 0.12427647655345161), (262, 0.12268102046117293), (128, 0.12130199878288984), (517, 0.12069079893127639), (2377, 0.11939641

In [28]:
print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1


Movies suggested for you : 

1 . Sita Ramam


IndexError: index 0 is out of bounds for axis 0 with size 0

In [29]:
print('Movies suggested for you : \n')
i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    titles_from_index = movies_data[movies_data['index'] == index]['title'].values

    if titles_from_index.size > 0:
        title_from_index = titles_from_index[0]
        if i < 30:
            print(i, '.', title_from_index)
            i += 1
    else:
        print(f"No title found for index {index}.")


Movies suggested for you : 

1 . sanam teri kasam
No title found for index 199.
No title found for index 12.
No title found for index 536.
No title found for index 1652.
No title found for index 384.
No title found for index 13.
No title found for index 129.
No title found for index 3055.
No title found for index 3033.
No title found for index 329.
No title found for index 178.
No title found for index 27.
No title found for index 2035.
No title found for index 267.
No title found for index 1183.
No title found for index 326.
No title found for index 1095.
No title found for index 1890.
No title found for index 2401.
No title found for index 2040.
No title found for index 2474.
No title found for index 2254.
No title found for index 1577.
No title found for index 894.
No title found for index 1061.
No title found for index 189.
No title found for index 2515.
No title found for index 2815.
No title found for index 1063.
No title found for index 1635.
No title found for index 840.
No tit

In [31]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Check the current working directory
print("Current working directory:", os.getcwd())

movies_data = pd.read_csv(r'Downloads/movies.csv')
print(movies_data.head())

# Display the shape
print("Shape of the dataset:", movies_data.shape)

# Selecting the relevant features for recommendation
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
print("Selected features:", selected_features)

# Display the info
print("Dataset info:")
print(movies_data.info())

# Check for Null Values
print("Null values in the dataset:")
print(movies_data.isna().sum())

# Display the selected columns
print("Selected columns:")
print(movies_data[selected_features].head())

# Check Null Values in selected columns
print("Null values in selected columns:")
print(movies_data[selected_features].isna().sum())

# Replacing the null values with empty strings
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')
print("Data after filling null values:")
print(movies_data.head())

# Combining all the 5 selected features
combined_features = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']
print("Combined features:")
print(combined_features.head())

# Converting the text data to feature vectors
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
print("Feature vectors shape:", feature_vectors.shape)
print("Feature vectors:")
print(feature_vectors)

# Getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)
print("Cosine similarity scores:")
print(similarity)

# Print Shape Cosine Similarity
print("Cosine similarity shape:", similarity.shape)

# Enter the movie name to get Similarity
movie_name = input('Enter your favourite movie name: ')

# Creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
print("List of all movie titles:")
print(list_of_all_titles)

# Length of the List
print("Length of the list of all titles:", len(list_of_all_titles))

# Finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print("Close matches found:")
print(find_close_match)

# Check if any close matches are found
if not find_close_match:
    print("No close matches found for the movie name entered.")
else:
    close_match = find_close_match[0]
    print("Close match selected:", close_match)

    # Finding the index of the movie with the title
    index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values
    if index_of_the_movie.size == 0:
        print("No index found for the matched movie.")
    else:
        index_of_the_movie = index_of_the_movie[0]
        print("Index of the matched movie:", index_of_the_movie)

        # Get the similarity row for the selected index
        similarity_score = list(enumerate(similarity[index_of_the_movie]))
        print("Similarity scores:")
        print(similarity_score)

        # Length of Similarity Score
        print("Length of similarity scores:", len(similarity_score))

        # Sorting the movies based on their similarity score
        sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        print("Sorted similar movies:")
        print(sorted_similar_movies)

        # Print the name of similar movies based on the index – Top 30
        print('Movies suggested for you:\n')
        i = 1
        for movie in sorted_similar_movies:
            index = movie[0]
            titles_from_index = movies_data[movies_data['index'] == index]['title'].values
            if titles_from_index.size > 0:
                title_from_index = titles_from_index[0]
                if i < 30:
                    print(f"{i}. {title_from_index}")
                    i += 1

# Consolidated code
def get_movie_recommendations(movie_name, movies_data, similarity):
    list_of_all_titles = movies_data['title'].tolist()
    find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
    
    if not find_close_match:
        return "No close matches found for the movie name entered."
    
    close_match = find_close_match[0]
    index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values
    if index_of_the_movie.size == 0:
        return "No index found for the matched movie."
    
    index_of_the_movie = index_of_the_movie[0]
    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    
    recommendations = ['Movies suggested for you:\n']
    i = 1
    for movie in sorted_similar_movies:
        index = movie[0]
        titles_from_index = movies_data[movies_data['index'] == index]['title'].values
        if titles_from_index.size > 0:
            title_from_index = titles_from_index[0]
            if i < 30:
                recommendations.append(f"{i}. {title_from_index}")
                i += 1
    return "\n".join(recommendations)

# Example usage
movie_name = input('Enter your favourite movie name: ')
recommendations = get_movie_recommendations(movie_name, movies_data, similarity)
print(recommendations)


Current working directory: C:\Users\divya
   index     budget                                    genres  \
0      0  237000000  Action Adventure Fantasy Science Fiction   
1      1  300000000                  Adventure Fantasy Action   
2      2  245000000                    Action Adventure Crime   
3      3  250000000               Action Crime Drama Thriller   
4      4  260000000          Action Adventure Science Fiction   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  culture clash future space war space colony so...                en   
1  ocean drug abuse exotic island east india trad...            